In [56]:
%%writefile app.py
import streamlit as st
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense,Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ModelCheckpoint 
import datetime
import re
import pickle
#!pip install -U ipykernel
from math import log
from numpy import array
from numpy import argmax
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

st.title(" PREDICTING CORRECT FORM OF THE SENTENCE!")




input_sentence=st.text_input('Enter The Corrupted Text')

#st.title(input_sentence)

submit=st.button("TRANSLATE!")

if submit:

  with st.spinner(text="This may take a moment..."):
    class Encoder(tf.keras.Model):
   
      def __init__(self,inp_vocab_size,embedding_size,lstm_size,input_length):
        super().__init__()
        #Initialize Embedding layer
        #Intialize Encoder LSTM layer
        self.embedding = Embedding(input_dim=inp_vocab_size, output_dim=embedding_size, input_length=input_length,
                           mask_zero=True, name="embedding_layer_encoder")
        self.lstm = LSTM(lstm_size, return_state=True, return_sequences=True, name="Encoder_LSTM")

      def call(self,input_sequence,initial_state):
         
        input_embedd                           = self.embedding(input_sequence)
        self.lstm_output, self.state_h,self.state_c = self.lstm(input_embedd)
        return self.lstm_output, self.state_h,self.state_c

    
      def initialize_states(self,batch_size):
      
        self.state_h=tf.zeros([batch_size])
        self.state_c=tf.zeros([batch_size])
        return self.state_h,self.state_c

    
    class Attention(tf.keras.layers.Layer):
  
      def __init__(self,scoring_function, att_units):

        super().__init__()
        self.att_units=att_units
        self.scoring_function=scoring_function
    # Please go through the reference notebook and research paper to complete the scoring functions

        if self.scoring_function=='dot':
      # Intialize variables needed for Dot score function here
      
          pass
        if scoring_function == 'general':
      # Intialize variables needed for General score function here
          self.W=Dense(self.att_units)  
      
          pass
        elif scoring_function == 'concat':
      # Intialize variables needed for Concat score function here
          self.W1=Dense(self.att_units,activation='tanh')
          self.W2=Dense(self.att_units,activation='tanh')  
    
          self.V=Dense(1)
          pass
  
  
      def call(self,decoder_hidden_state,encoder_output):
    
    
        if self.scoring_function == 'dot':
        # Implement Dot score function here
        #print(encoder_output.shape,decoder_hidden_state.shape)
          decoder_hidden_state=tf.reshape(decoder_hidden_state,shape=(tf.shape(decoder_hidden_state)[0],1,tf.shape(decoder_hidden_state)[1]))
        #print(decoder_hidden_state.shape[0])
        #print(decoder_hidden_state.shape[1])
        #print(decoder_hidden_state.shape)
          score=tf.matmul(decoder_hidden_state,encoder_output,transpose_b=True)
        
        
        elif self.scoring_function == 'general':
        # Implement General score function here
          decoder_hidden_state=tf.reshape(decoder_hidden_state,shape=(tf.shape(decoder_hidden_state)[0],1,tf.shape(decoder_hidden_state)[1]))
          score=tf.matmul(decoder_hidden_state,self.W(encoder_output),transpose_b=True)
       

        elif self.scoring_function == 'concat':
        # Implement General score function here
          decoder=tf.expand_dims(decoder_hidden_state,1)
          score=self.V(tf.nn.tanh(self.W1(decoder)+self.W2(encoder_output)))
          attention_weights=tf.nn.softmax(score,axis=1)
          context_vector=attention_weights*encoder_output
          context_vector=tf.reduce_sum(context_vector,axis=1)

          return context_vector, attention_weights

        
        attention_weights=tf.nn.softmax(score,axis=2)
        context_vector=tf.matmul(attention_weights,encoder_output)
        context_vector=tf.reshape(context_vector,shape=(tf.shape(context_vector)[0],tf.shape(context_vector)[2]))
        attention_weights=tf.reshape(attention_weights,shape=(tf.shape(attention_weights)[0],tf.shape(attention_weights)[2],tf.shape(attention_weights)[1]))
        return context_vector, attention_weights
    
    class One_Step_Decoder(tf.keras.Model):
      def __init__(self,tar_vocab_size, embedding_dim, input_length, dec_units ,score_fun ,att_units):
        super().__init__()
      # Initialize decoder embedding layer, LSTM and any other objects needed
        self.embedding = Embedding(input_dim=tar_vocab_size, output_dim=embedding_dim, input_length=input_length,
                           mask_zero=True, name="embedding_layer")
        self.lstm=LSTM(dec_units,return_sequences=True,return_state=True)
        self.attention=Attention(score_fun,att_units)
        self.dense=tf.keras.layers.Dense(tar_vocab_size)


      def call(self,input_to_decoder, encoder_output, state_h,state_c):
   
        x=self.embedding(input_to_decoder)
        context_vector,attention_weights=self.attention(state_h,encoder_output)
        concat=tf.concat([x,tf.expand_dims(context_vector,1)],axis=-1)
        output,decoder_state_h,decoder_state_c=self.lstm(x,initial_state=[state_h,state_c])
        output=tf.reshape(output,(-1,output.shape[2]))
        final_output=self.dense(output)
        return final_output,decoder_state_h,decoder_state_c,attention_weights,context_vector

    class Decoder(tf.keras.Model):
      def __init__(self,out_vocab_size, embedding_dim, input_length, dec_units ,score_fun ,att_units):
      #Intialize necessary variables and create an object from the class onestepdecoder
        super(Decoder,self).__init__()
        self.out_vocab_size=out_vocab_size
        self.embedding_dim=embedding_dim
        self.input_length=input_length
        self.dec_units=dec_units
        self.score_fun=score_fun
        self.att_units=att_units
        self.One_Step_Decoder=One_Step_Decoder(self.out_vocab_size, self.embedding_dim, self.input_length, self.dec_units ,self.score_fun ,self.att_units)
        
      def call(self, input_to_decoder,encoder_output,decoder_hidden_state,decoder_cell_state ):

        #Initialize an empty Tensor array, that will store the outputs at each and every time step
        #Create a tensor array as shown in the reference notebook
        
        #Iterate till the length of the decoder input
            # Call onestepdecoder for each token in decoder_input
            # Store the output in tensorarray
        # Return the tensor array
          all_outputs=tf.TensorArray(tf.float32,size=tf.shape(input_to_decoder)[1])
          for i in range(tf.shape(input_to_decoder)[1]):
            output,decoder_hidden_state,decoder_cell_state,attention_weights,context_vector=self.One_Step_Decoder(input_to_decoder[:,i:i+1],encoder_output,decoder_hidden_state,decoder_cell_state)
          
            all_outputs=all_outputs.write(i,output)
          all_outputs=tf.transpose(all_outputs.stack(),[1,0,2])
          return all_outputs

    class encoder_decoder(tf.keras.Model):
      def __init__(self,inp_vocab_size, embedding_dim,enc_units,input_length,out_vocab_size,dec_units ,score_fun ,att_units,batch_size):
    #Intialize objects from encoder decoder
        super().__init__()
        self.batch_size=batch_size
        self.encoder=Encoder(inp_vocab_size,embedding_dim,enc_units,input_length)
        self.decoder=Decoder(out_vocab_size, embedding_dim, input_length, dec_units ,score_fun ,att_units)

  
      def call(self,data):
    #Intialize encoder states, Pass the encoder_sequence to the embedding layer
    # Decoder initial states are encoder final states, Initialize it accordingly
    # Pass the decoder sequence,encoder_output,decoder states to Decoder
    # return the decoder output
    
        input,output=data[0],data[1]
        initial_state=self.encoder.initialize_states(self.batch_size)
        encoder_output,encoder_final_state_h,encoder_final_state_c=self.encoder(input,initial_state)
        decoder_outputs=self.decoder(output,encoder_output,encoder_final_state_h,encoder_final_state_c)

        return decoder_outputs

    def load_models():
  
      train=pd.read_pickle('train.pkl')
      validation=pd.read_pickle('validation.pkl')

      with open('tok_corrupted.pickle', 'rb') as handle:
        tok_corrupted = pickle.load(handle)


      with open('tok_uncorrupted.pickle', 'rb') as handle1:
        tok_uncorrupted = pickle.load(handle1)


      vocab_corrupt=len(tok_corrupted.word_index.keys())
      vocab_uncorrupt=len(tok_uncorrupted.word_index.keys())

      train_dataset = Dataset(train, tok_corrupted, tok_uncorrupted,25 )
      test_dataset  = Dataset(validation,tok_corrupted, tok_uncorrupted,25)


      train_dataloader = Dataloder(train_dataset, batch_size=64)
      test_dataloader = Dataloder(test_dataset, batch_size=64)


      model  = encoder_decoder(vocab_corrupt,100,300,25,vocab_uncorrupt,300,'concat',50,64)
      optimizer = tf.keras.optimizers.Adam()
      model.compile(optimizer=optimizer,loss=loss_function,metrics=[accuracy])
      batch_size=64
  
      model.fit_generator(train_dataloader, steps_per_epoch=1, epochs=1, validation_data=test_dataloader, validation_steps=1)
      model.load_weights('model.h5')

      return model

    def loss_function(real, pred):
    
    
    
      mask = tf.math.logical_not(tf.math.equal(real, 0))
      loss_ = loss_object(real, pred)

      mask = tf.cast(mask, dtype=loss_.dtype)
      loss_ *= mask

      return tf.reduce_mean(loss_)
    #@st.cache
    class Dataset:
      def __init__(self, data, tok_corrupt, tok_uncorrupt, max_len):
        self.encoder_inps = data['corrupted'].values
        self.decoder_inps = data['uncorrupted_inp'].values
        self.decoder_outs = data['uncorrupted_op'].values
        self.tok_corrupt = tok_corrupt
        self.tok_uncorrupt = tok_uncorrupt
        self.max_len = max_len

      def __getitem__(self, i):
        self.encoder_seq = self.tok_corrupt.texts_to_sequences([self.encoder_inps[i]]) # need to pass list of values
        self.decoder_inp_seq = self.tok_uncorrupt.texts_to_sequences([self.decoder_inps[i]])
        self.decoder_out_seq = self.tok_uncorrupt.texts_to_sequences([self.decoder_outs[i]])

        self.encoder_seq = pad_sequences(self.encoder_seq, maxlen=self.max_len, dtype='int32', padding='post')
        self.decoder_inp_seq = pad_sequences(self.decoder_inp_seq, maxlen=self.max_len, dtype='int32', padding='post')
        self.decoder_out_seq = pad_sequences(self.decoder_out_seq, maxlen=self.max_len, dtype='int32', padding='post')
        return self.encoder_seq, self.decoder_inp_seq, self.decoder_out_seq

      def __len__(self): # your model.fit_gen requires this function
        return len(self.encoder_inps)

    
    class Dataloder(tf.keras.utils.Sequence):    
      def __init__(self, dataset, batch_size=1):
        self.dataset = dataset
        self.batch_size = batch_size
        self.indexes = np.arange(len(self.dataset.encoder_inps))

      def __getitem__(self, i):
        start = i * self.batch_size
        stop = (i + 1) * self.batch_size
        data = []
        for j in range(start, stop):
            data.append(self.dataset[j])

        batch = [np.squeeze(np.stack(samples, axis=1), axis=0) for samples in zip(*data)]
        return tuple([[batch[0],batch[1]],batch[2]])

      def __len__(self):  # your model.fit_gen requires this function
        return len(self.indexes) // self.batch_size

      def on_epoch_end(self):
        self.indexes = np.random.permutation(self.indexes)


    def beam_search_decoder(data, k):
      sequences = [[list(), 0.0]]
      for row in data:
        all_candidates = list()
        for i in range(len(sequences)):
          seq, score = sequences[i]
          for j in range(len(row)):
            try:
              candidate = [seq + [j], score - log(row[j])]
              all_candidates.append(candidate)
            except ValueError as e:
              candidate = [seq + [j], 0]
              all_candidates.append(candidate)
      # order all candidates by score
        ordered = sorted(all_candidates, key=lambda tup:tup[1])
    # select k best
        sequences = ordered[:k]
      return sequences

    from tensorflow.keras import backend as K
    def accuracy(y_true, y_pred):

      pred_value= K.cast(K.argmax(y_pred, axis=-1), dtype='float32')
      true_value = K.cast(K.equal(y_true, pred_value), dtype='float32')

      mask = K.cast(K.greater(y_true, 0), dtype='float32')
      n_correct = K.sum(mask * true_value)
      n_total = K.sum(mask)
  
      return n_correct / n_total

    def prediction(input_sentence):


      with open('tok_uncorrupted.pickle', 'rb') as handle1:
        tok_uncorrupted = pickle.load(handle1)

      with open('tok_corrupted.pickle', 'rb') as handle:
        tok_corrupted = pickle.load(handle)
  
      model=load_models()

  
 
      input_vec = tok_corrupted.texts_to_sequences([input_sentence])
      input_vec =  tf.keras.preprocessing.sequence.pad_sequences(input_vec,maxlen=25,padding='post')
      input_vec = tf.convert_to_tensor(input_vec)
      enc_initial_states = None
      enc_out, h_state, c_state = model.layers[0](input_vec,enc_initial_states)
      curr_vec = np.array(tok_uncorrupted.word_index["<start>"])
      curr_vec = curr_vec.reshape(1,1)
      end_vec = np.array(tok_uncorrupted.word_index["<end>"]).reshape(1,1)
      result=''
      atten_weights= np.zeros((25,25)) #(max_target_length, max_source_length)
      for i in range(25):
        decoder_output, lstm_state_h, lstm_state_c, atten_w, context_vectors = model.layers[1].One_Step_Decoder(curr_vec, enc_out, h_state, c_state)
    
        Result_beam_list=beam_search_decoder(decoder_output,k=1)
        Result_beam=Result_beam_list[0][0]
   
        predicted_id = tf.argmax(decoder_output[0]).numpy()
      
    
        if tok_uncorrupted.index_word[predicted_id] != '<end>':
          result += tok_uncorrupted.index_word[Result_beam[0]] + ' '
      # the predicted ID is fed back into the model
      
        h_state, c_state = lstm_state_h, lstm_state_c
        curr_vec = tf.expand_dims([predicted_id], 0)
  

      return result
    #result, input_sentence, attention_plot = predict(inp_sentence)
    #print('Input: %s' % (input_sentence))
    #print("-"*50)
    ##print('Predicted translation: {}'.format(result))
    #print("-"*50)
    
    #return result

  st.text_area('Translated Sentence:',prediction(input_sentence))
  

st.markdown('''
    <a href="https://www.linkedin.com/in/abhishek-rai-8b3634169/">
        <img src="data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAGQAAABkCAMAAABHPGVmAAAAaVBMVEUAd7X///8AcLEEebb7/f4Debb9/v79/v8DeLY8l8Y7lsYAbbCcyuKlz+WQxN+Lwd0KfbiDvdvl8fcAc7O+3OwgiL6x1ehFnMlytNZqsNQZhLzF4O7w9/tSo81gqtGp0ebc7PUsjsJ8udngaOvIAAADg0lEQVRoge2abZeqIBCAGV7EjXItKjc1y/7/j7xD7m5AWHdD+uScs3s8kDzOIMwMDoGrSJ4lEC6H0ckVkUEiyeQPBC+ar+pjcqm+GsQMEAntUSuaQJQ+toZC8F+7U4KJBMKE6ltUwkAqJUgiEapCAJGwpSwVgxBGtyAJh4NKCVEH4AStRZNZC+1F0V5orgXN00FyukBzZekh2XsgbzFXCMIGSQlhTCsjegrMCEQQVXfF8rOrFYm3YxjCdFnw6zbdFkJHr6AgROjdGYAbATjXOlaXEISRcgN88GEZh3NJIuclBBF0BfzXr3EoYve1ACTXPViuOAO+izRYAMLUxVLEqBK7RQcggp48yHJ6SG5cjAWRsI/0A0FN9g6Ew2p6TZhae+a6pDDXwn67UKt++rcL3+Gts072sW4gDOn5L4VjRBa7rwT3LkarDMNvmeEf8EV0tDSyC9O++dZku4uPyEb8CdO6KrbNtvjQOt5rjXnGnGGkrKmibALfP+7jBQ6fs0lCvnHIn4Tlj2KBsTkJX4cGYgLH1yiD8n+A/LcgACevrHe7ujRXQYXCEFHaIsLNZjgMnMrjstm0nLebZnXodSi6Cbvfz3Zzk3Y97I/ozOxmE2Dkql5vnCS0OQbe+WAgoZbOjcUPpLBbeU0I7dprZi4xlZYyM8ENNP3d6h2DcPkj/Ab5vDVjkllrgg/DpfNA+HN59D3DKMTahW3ILbxoa3Ru/D75x59Uni6vQ3hZOL7N8j9+ePMqRMLmACNnGHce6FUItrcgA4ThCdwU9HUIjDLuVImAjB/4+EFnjCZXba6xv/SU8oLOOEhmvQgu5GRPShQER25Pl667nFrXeBmc7ZUSA8FxLyU1mSWtTw7Fm5S4ia/Ud5KsnVtM34dlrwiIxNkV31ZhhG1sXTh01sy/DkG7W55SqC9npcLXJBBvHEzPLFU4rCeBgBeHs7MDKaaA4A5ZWggvdXKzs5chGTSOn2XKTZlX02jiJnnGOU8O8ZO8u146CWTpQdYzZIbMkBkyQ2bIDJkhA+QmDsRqDkCs3tXTqH4FtiyDxx6BkMiW/ZO4K6eH/eom+27oy2nnNB+8DPfo9j5JgswBk/PVnj1sfnzTKMScPVpf7fMnzU97JzqDfCxvhMxfsf8Ake8pX3hLIUbqkhJtSkpSF8ccTXGMKSXqk5f5vKVg6Xpkkbz06j1FZJC6HO4fbRVnSvd35FUAAAAASUVORK5CYII=" />
    </a>''',
    unsafe_allow_html=True
)

Overwriting app.py


In [14]:
!pip install -q pyngrok

     |████████████████████████████████| 745 kB 7.4 MB/s 


In [19]:
!ngrok authtoken 23Uk3EbRKXFi66Ci23h6OjE1Uck_eikTGoay8PBxbVo6JBX7

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [57]:
from pyngrok import ngrok
public_url = ngrok.connect(port='80')
print (public_url)
!streamlit run --server.port 80 app.py >/dev/null

2022-01-13 08:23:41.920 INFO    pyngrok.ngrok: Opening tunnel named: http-80-be8e3363-3453-44f3-bf35-661af3541e96
2022-01-13 08:23:41.959 INFO    pyngrok.process.ngrok: t=2022-01-13T08:23:41+0000 lvl=info msg="no configuration paths supplied"
2022-01-13 08:23:41.961 INFO    pyngrok.process.ngrok: t=2022-01-13T08:23:41+0000 lvl=info msg="using configuration at default config path" path=/root/.ngrok2/ngrok.yml
2022-01-13 08:23:41.971 INFO    pyngrok.process.ngrok: t=2022-01-13T08:23:41+0000 lvl=info msg="open config file" path=/root/.ngrok2/ngrok.yml err=nil
2022-01-13 08:23:41.972 INFO    pyngrok.process.ngrok: t=2022-01-13T08:23:41+0000 lvl=info msg="starting web service" obj=web addr=127.0.0.1:4040
2022-01-13 08:23:42.190 INFO    pyngrok.process.ngrok: t=2022-01-13T08:23:42+0000 lvl=info msg="tunnel session started" obj=tunnels.session
2022-01-13 08:23:42.192 INFO    pyngrok.process.ngrok: t=2022-01-13T08:23:42+0000 lvl=info msg="client session established" obj=csess id=07b6aa9f503d
2

NgrokTunnel: "http://41a4-34-125-196-62.ngrok.io" -> "http://localhost:80"


2022-01-13 08:23:47.417 INFO    pyngrok.process.ngrok: t=2022-01-13T08:23:47+0000 lvl=info msg="join connections" obj=join id=01f08208b49d l=127.0.0.1:80 r=124.108.16.134:51066
2022-01-13 08:23:48.832 INFO    pyngrok.process.ngrok: t=2022-01-13T08:23:48+0000 lvl=info msg="join connections" obj=join id=b762dce3f8e9 l=127.0.0.1:80 r=124.108.16.134:50786
2022-01-13 08:23:48.837 INFO    pyngrok.process.ngrok: t=2022-01-13T08:23:48+0000 lvl=info msg="join connections" obj=join id=8110813c16cf l=127.0.0.1:80 r=124.108.16.134:57020
2022-01-13 08:23:48.858 INFO    pyngrok.process.ngrok: t=2022-01-13T08:23:48+0000 lvl=info msg="join connections" obj=join id=39254534b280 l=127.0.0.1:80 r=124.108.16.134:57022
2022-01-13 08:23:48.897 INFO    pyngrok.process.ngrok: t=2022-01-13T08:23:48+0000 lvl=info msg="join connections" obj=join id=04946d74fd75 l=127.0.0.1:80 r=124.108.16.134:51070
2022-01-13 08:23:48.923 INFO    pyngrok.process.ngrok: t=2022-01-13T08:23:48+0000 lvl=info msg="join connections" o

2022-01-13 08:24:10.772167: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
/content/app.py:224: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_dataloader, steps_per_epoch=1, epochs=1, validation_data=test_dataloader, validation_steps=1)
/content/app.py:224: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_dataloader, steps_per_epoch=1, epochs=1, validation_data=test_dataloader, validation_steps=1)


2022-01-13 08:26:36.547 INFO    pyngrok.process.ngrok: t=2022-01-13T08:26:36+0000 lvl=info msg="received stop request" obj=app stopReq="{err:<nil> restart:false}"
2022-01-13 08:26:36.551 INFO    pyngrok.process.ngrok: t=2022-01-13T08:26:36+0000 lvl=info msg="session closing" obj=tunnels.session err=nil
